# **Training on PBMC**
<a target="_blank" href="https://colab.research.google.com/github/raphaelrubrice/scVAE_mva2025/blob/raph/data_pipeline.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## **Colab setup**

In [1]:
!git clone https://github.com/raphaelrubrice/scVAE_mva2025.git
%cd scVAE_mva2025
!python -m pip install -r requirements.txt
!git checkout raph

Cloning into 'scVAE_mva2025'...
remote: Enumerating objects: 287, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 287 (delta 85), reused 90 (delta 42), pack-reused 131 (from 1)
Receiving objects: 100% (287/287), 972.15 KiB | 29.46 MiB/s, done.
Resolving deltas: 100% (149/149), done.
/content/scVAE_mva2025
Branch 'raph' set up to track remote branch 'raph' from 'origin'.
Switched to a new branch 'raph'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


### Test Downloads

In [2]:
from data_pipeline.src.downloader import run_downloads

run_downloads()


Saving extracted PBMC datasets to: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw

⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34


CD34: 100%|██████████| 38.0M/38.0M [00:02<00:00, 14.5MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD34
✓ Finished CD34
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B


CD19_B: 100%|██████████| 18.0M/18.0M [00:01<00:00, 13.3MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD19_B
✓ Finished CD19_B
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD56_NK


...

✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD4_CD45RO
✓ Finished CD4_CD45RO
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8


CD8: 100%|██████████| 20.0M/20.0M [00:01<00:00, 10.8MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8
✓ Finished CD8
------------------------------------------------------------
⇩ Downloading and extracting to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA


CD8_CD45RA: 100%|██████████| 20.9M/20.9M [00:01<00:00, 13.1MB/s]


✓ Extracted to /content/scVAE_mva2025/data_pipeline/data/pbmc_raw/CD8_CD45RA
✓ Finished CD8_CD45RA
------------------------------------------------------------
All PBMC datasets ready in: /content/scVAE_mva2025/data_pipeline/data/pbmc_raw


In [3]:
from data_pipeline.src.config import DATASETS
from data_pipeline.src.load_anndata import load_anndata
from pathlib import Path

folder = Path("data_pipeline/data/pbmc_raw/CD4_CD45RO")
meta = DATASETS["CD4_CD45RO"]

adata = load_anndata(folder, meta)
adata

ModuleNotFoundError: No module named 'scanpy'

In [ ]:
from data_pipeline.src.combine import run_combine

combined, collection = run_combine(
    do_write_shards=True,
    write_combined=True,
    harmonize_var=False
)

In [ ]:
import scanpy as sc
adata_combined = sc.read_h5ad("data_pipeline/data/pbmc_processed/pbmc_combined.h5ad")
adata_combined

In [ ]:
from data_pipeline.src.dataloader import build_cv_dataloaders

N_GENES = 500 # Authors used 5000 most variation, we can take 10% of that
kept_idx, folds, test_loader = build_cv_dataloaders(
    shard_dir="data_pipeline/data/pbmc_processed/shards",
    label_maps_path="data_pipeline/data/pbmc_processed/label_maps.json",
    batch_size=256,
    one_hot=True,
    pin_m=True,
    filter_genes=True,
    max_genes=N_GENES,
    downsample=True, # take 10% of the dataset
    frac=0.1
)

# **Cross-Validation of architectures**

In [ ]:
import torch
from tqdm.auto import tqdm
from mixture_vae.run_handling import run_cv
from mixture_vae.viz import plot_loss_components, plot_latent

In [ ]:
# Problem setup
input_dim = N_GENES # genes
hidden_dim = 100 # hidden neurons per layer
n_components = 9 # 9 clusters are assumed
latent_dim = 100 # dimension latent space
n_layers = 2 # number of encoding and decoding layers

LR = 1e-3
EPOCHS = 50
beta_kl = 1
warmup = int(0.4*EPOCHS)

## **Cross-Validated scVAE**

In [ ]:
from mixture_vae.mvae import MixtureVAE

In [ ]:
# model + priorlatent + prior_input + posterior latent (we will always use uniform prior categorical ?)
save_path = "./scVAE_Normal_NB_Normal.ckpt"

In [ ]:
config_mvae = {
        # Run name and model
        "run_tag": "scVAE_Normal_NB_Normal",
        "model_type": "MixtureVAE",

        # priors and posteriors
        "prior_latent_dist": "Normal",
        "prior_input_dist": "NegativeBinomial",
        "posterior_latent_dist": "Normal",
        "prior_categorical_dist": "Uniform",

        # architecture params
        "input_dim": input_dim,
        "hidden_dim": hidden_dim,
        "latent_dim": latent_dim,
        "n_components": n_components,
        "n_layers": n_layers,

        # training params
        "lr": 1e-3,
        "epochs": EPOCHS,
        "beta_kl": beta_kl,
        "warmup": warmup,
        "save_path": save_path
    }

In [ ]:
cv_mvae, mvae_metrics = run_cv(config_mvae, folds, test_loader)

## **Cross-Validated independent Mixture of Mixtures**

In [ ]:
from mixture_vae.mvae import ind_MoMVAE

In [ ]:
# model + priorlatent + prior_input + posterior latent (we will always use uniform prior categorical ?)
save_path = "./Ind_MoM_Normal_NB_Normal.ckpt"

In [ ]:
config_indmom = {
        # Run name and model
        "run_tag": "Ind_MoM_Normal_NB_Normal",
        "model_type": "ind_MoMVAE",

        # priors and posteriors
        "prior_latent_dist": "Normal",
        "prior_input_dist": "NegativeBinomial",
        "posterior_latent_dist": "Normal",
        "prior_categorical_dist": "Uniform",

        # architecture params
        "input_dim": input_dim,
        "hidden_dim": hidden_dim,
        "latent_dim": latent_dim,
        "hierarchy_components": [2, 3, 5, 9],
        "n_layers": n_layers,

        # training params
        "lr": 1e-3,
        "epochs": EPOCHS,
        "beta_kl": beta_kl,
        "warmup": warmup,
        "save_path": save_path
    }

In [ ]:
cv_indmom, indmom_metrics = run_cv(config_indmom, folds, test_loader)

## **Cross-Validated Mixture of Mixtures**

In [ ]:
from mixture_vae.mvae import MoMixVAE

In [ ]:
# model + priorlatent + prior_input + posterior latent (we will always use uniform prior categorical ?)
save_path = "./MoMix_Normal_NB_Normal.ckpt"

In [ ]:
config_momix = {
        # Run name and model
        "run_tag": "MoMix_Normal_NB_Normal",
        "model_type": "MoMixVAE",

        # priors and posteriors
        "prior_latent_dist": "Normal",
        "prior_input_dist": "NegativeBinomial",
        "posterior_latent_dist": "Normal",
        "prior_categorical_dist": "Uniform",

        # architecture params
        "input_dim": input_dim,
        "hidden_dim": hidden_dim,
        "latent_dim": latent_dim,
        "hierarchy_components": [2, 3, 5, 9],
        "n_layers": n_layers,

        # training params
        "lr": 1e-3,
        "epochs": EPOCHS,
        "beta_kl": beta_kl,
        "warmup": warmup,
        "save_path": save_path
    }

In [ ]:
cv_momix, momix_metrics = run_cv(config_momix, folds, test_loader)

# **Figure 1: Generative and Clustering performances**

In [ ]:
from mixture_vae.figure1 import plot_figure1

In [ ]:
def extract_models(res_list):
    return [dico["model"] for dico in res_list]

In [ ]:
cv_scVAE = extract_models(cv_mvae)
cv_IndMoM = extract_models(cv_indmom)
cv_MoMix = extract_models(cv_momix)

In [ ]:
cv_dico = {"$\mathcal{N}$(0,I)": [cv_scVAE, cv_IndMoM, cv_MoMix]}

plot_figure1(cv_dico, test_loader, save_path="./Figure1.pdf")